In [472]:
if 'google.colab' in str(get_ipython()):
    !pip uninstall tensorflow -y
    !pip install tensorflow==2.3.1 tensorflow-quantum neptune-client
    !rm -rf quantum-gans
    !git clone https://github.com/WiktorJ/quantum-gans
    !cd quantum-gans; pip install .
    neptun_token = "" # put manually for the time being
else:
    import subprocess
    def get_var(varname):
        CMD = 'echo $(source ~/.bash_profile; echo $%s)' % varname
        p = subprocess.Popen(CMD, stdout=subprocess.PIPE, shell=True, executable='/bin/bash')
        return p.stdout.readlines()[0].strip()
    neptun_token = get_var('NEPTUNE_API_TOKEN').decode("utf-8") 

In [473]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [474]:
import io
import math
import json
import neptune
import tensorflow as tf

import cirq
import numpy as np
from qsgenerator.utils import map_to_radians
from qsgenerator.qugans import circuits
from qsgenerator.resolvers import resolve_all
from qsgenerator.qugans.training import Trainer
from qsgenerator.phase.circuits import build_ground_state_circuit
from qsgenerator.phase.constants import z1_theta, z2_theta
from qsgenerator.phase.analitical import  get_ground_state_for_g
from qsgenerator.states.simple_state_circuits import build_x_rotation_state
from qsgenerator.states.simple_rotation_generators import get_binary_x_rotation_provider, get_arcsin_x_rotation_provider
from qsgenerator.phase.analitical import construct_hamiltonian, get_theta_v, get_theta_w, get_theta_r, get_g_parameters_provider
from qsgenerator.evaluators.circuit_evaluator import CircuitEvaluator

In [475]:
use_neptune = False

In [476]:
generator_layers = 5
discriminator_layers = 5
data_bus_size = 5

In [477]:
real_phase = True
generic_generator = True
all_layers_labeling = False
full_layer_labeling = False
use_gen_label_qubit = False
use_disc_label_qubit = False
zxz = False
fixed_gates = {}
# fixed_gates = {"z1": z1_theta}

In [478]:
gen, gs, disc, ds, ls, data_qubits, out_qubit = circuits.build_gan_circuits(
    generator_layers, 
    discriminator_layers, 
    data_bus_size, 
    all_layers_labeling=all_layers_labeling,
    full_layer_labeling=full_layer_labeling,
    use_gen_label_qubit=use_gen_label_qubit,
    use_disc_label_qubit=use_disc_label_qubit)

In [479]:
if not generic_generator:
    gen, gs, symbols_dict_gen = build_ground_state_circuit(qubits=data_qubits, full_parametrization=True, zxz=zxz)
else:
    symbols_dict_gen = {}

In [480]:
gen, gs = resolve_all(gen, gs, symbols_dict_gen, fixed_gates)

In [481]:
if real_phase:
    real, real_symbols, symbols_dict_real = build_ground_state_circuit(qubits=data_qubits)
else:
    real, real_symbols, symbols_dict_real = build_x_rotation_state(qubits=data_qubits)

In [482]:
pure_gen = gen.copy()
gen.append([disc])

In [483]:
pure_real = real.copy()
real.append([disc])

In [484]:
print("REAL GROUND STATE")
pure_real

REAL GROUND STATE


(0, 1): ───H───@─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
               │
(0, 2): ───────X───Z───Ry(theta_r)───X─────────────@───X───────────────────────────────────@───X─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                   │                                       │
(0, 3): ─────────────────────────────Ry(theta_w)───X───X───Ry(theta_w)───X───Ry(theta_v)───X───X───Ry(theta_v)───X───X─────────────@───X───────────────────────────────────@───X─────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                   │                                       │
(0, 4): ─────────────────────────────────────────────────────────────────────────────────────────────────────────────Ry(theta_w)───X───X───Ry(theta_w)───X───Ry(theta_v)───X───X───Ry(theta_v)───X───X─────────────@───X───────────────────────────────────@───X─────────────────────
                                                                                                                                                                                                                   │                                       │
(0, 5): ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────Ry(theta_w)───X───X───Ry(theta_w)───X───Ry(theta_v)───X───X───Ry(theta_v)───X───

In [488]:
print("GENERATOR")
pure_gen

GENERATOR


(0, 1): ───Rx(g0)───Rz(g5)───ZZ────────────────Rx(g14)───Rz(g19)───ZZ────────────────Rx(g28)───Rz(g33)───ZZ────────────────Rx(g42)───Rz(g47)───ZZ────────────────Rx(g56)───Rz(g61)───ZZ────────────────
                             │                                     │                                     │                                     │                                     │
(0, 2): ───Rx(g1)───Rz(g6)───ZZ^g10───ZZ───────Rx(g15)───Rz(g20)───ZZ^g24───ZZ───────Rx(g29)───Rz(g34)───ZZ^g38───ZZ───────Rx(g43)───Rz(g48)───ZZ^g52───ZZ───────Rx(g57)───Rz(g62)───ZZ^g66───ZZ───────
                                      │                                     │                                     │                                     │                                     │
(0, 3): ───Rx(g2)───Rz(g7)───ZZ───────ZZ^g12───Rx(g16)───Rz(g21)───ZZ───────ZZ^g26───Rx(g30)───Rz(g35)───ZZ───────ZZ^g40───Rx(g44)───Rz(g49)───ZZ───────ZZ^g54───Rx(g58)───Rz(g63)───ZZ───────ZZ^g68───
                             │                                     │                                     │                                     │                                     │
(0, 4): ───Rx(g3)───Rz(g8)───ZZ^g11───ZZ───────Rx(g17)───Rz(g22)───ZZ^g25───ZZ───────Rx(g31)───Rz(g36)───ZZ^g39───ZZ───────Rx(g45)───Rz(g50)───ZZ^g53───ZZ───────Rx(g59)───Rz(g64)───ZZ^g67───ZZ───────
                                      │                                     │                                     │                                     │                                     │
(0, 5): ───Rx(g4)───Rz(g9)────────────ZZ^g13───Rx(g18)───Rz(g23)────────────ZZ^g27───Rx(g32)───Rz(g37)────────────ZZ^g41───Rx(g46)───Rz(g51)────────────ZZ^g55───Rx(g60)───Rz(g65)────────────ZZ^g69───

In [486]:
print("DISCRIMINATOR")
disc

DISCRIMINATOR


(0, 0): ───Rx(d0)───Rz(d6)────ZZ────────────────Rx(d17)───Rz(d23)───ZZ────────────────Rx(d34)───Rz(d40)───ZZ────────────────Rx(d51)───Rz(d57)───ZZ────────────────Rx(d68)───Rz(d74)───ZZ────────────────
                              │                                     │                                     │                                     │                                     │
(0, 1): ───Rx(d1)───Rz(d7)────ZZ^d12───ZZ───────Rx(d18)───Rz(d24)───ZZ^d29───ZZ───────Rx(d35)───Rz(d41)───ZZ^d46───ZZ───────Rx(d52)───Rz(d58)───ZZ^d63───ZZ───────Rx(d69)───Rz(d75)───ZZ^d80───ZZ───────
                                       │                                     │                                     │                                     │                                     │
(0, 2): ───Rx(d2)───Rz(d8)────ZZ───────ZZ^d15───Rx(d19)───Rz(d25)───ZZ───────ZZ^d32───Rx(d36)───Rz(d42)───ZZ───────ZZ^d49───Rx(d53)───Rz(d59)───ZZ───────ZZ^d66───Rx(d70)───Rz(d76)───ZZ───────ZZ^d83───
                              │                                     │                                     │                                     │                                     │
(0, 3): ───Rx(d3)───Rz(d9)────ZZ^d13───ZZ───────Rx(d20)───Rz(d26)───ZZ^d30───ZZ───────Rx(d37)───Rz(d43)───ZZ^d47───ZZ───────Rx(d54)───Rz(d60)───ZZ^d64───ZZ───────Rx(d71)───Rz(d77)───ZZ^d81───ZZ───────
                                       │                                     │                                     │                                     │                                     │
(0, 4): ───Rx(d4)───Rz(d10)───ZZ───────ZZ^d16───Rx(d21)───Rz(d27)───ZZ───────ZZ^d33───Rx(d38)───Rz(d44)───ZZ───────ZZ^d50───Rx(d55)───Rz(d61)───ZZ───────ZZ^d67───Rx(d72)───Rz(d78)───ZZ───────ZZ^d84───
                              │                                     │                                     │                                     │                                     │
(0, 5): ───Rx(d5)───Rz(d11)───ZZ^d14────────────Rx(d22)───Rz(d28)───ZZ^d31────────────Rx(d39)───Rz(d45)───ZZ^d48────────────Rx(d56)───Rz(d62)───ZZ^d65────────────Rx(d73)───Rz(d79)───ZZ^d82────────────

In [ ]:
np.random.seed(0)
eps = 1e-2
init_gen_weights = np.array([0] * len(gs)) + \
                   np.random.normal(scale=eps, size=(len(gs),))
init_disc_weights = np.random.normal(size=(len(ds),))

gen_weights = tf.Variable(init_gen_weights, dtype=tf.float32)
disc_weights = tf.Variable(init_disc_weights, dtype=tf.float32)

In [ ]:
class CustomScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, warmup_steps=4000):
        super(CustomScheduler, self).__init__()
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        return max(math.e ** - ((step+200) / (self.warmup_steps / math.log(100))), 0.01)

In [ ]:
learning_rate = CustomScheduler()

opt = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

In [ ]:
#g_values = [0.1, 0.4, 0.25]
g_values = [-0.5]
# x_rotations = get_binary_x_rotation_provider({0: '100', 1: '011', 2: '101'})
# real_values_provider = get_arcsin_x_rotation_provider(g_values, data_bus_size)
real_values_provider = get_g_parameters_provider()

In [ ]:
epochs = 3
disc_iteration = 1
gen_iteration = 1

In [ ]:
if use_neptune:
    neptune.init(project_qualified_name='wiktor.jurasz/thesis', api_token=neptun_token)
    neptun_params = {
        'generator_layers': generator_layers,
        'discriminator_layers': discriminator_layers,
        'g_values': g_values,
        'size':  data_bus_size,
        'real_phase': real_phase,
        'generic_generator': generic_generator,
        'all_layers_labeling': all_layers_labeling,
        'full_layer_labeling': full_layer_labeling,
        'use_gen_label_qubit': use_gen_label_qubit,
        'use_disc_label_qubit': use_disc_label_qubit,
        'zxz': zxz,
        'fixed_gates': fixed_gates,
        'disc_iteration': disc_iteration,
        'gen_iteration': gen_iteration,
        'epochs': epochs
    }
    neptune.create_experiment(name=None, description=None, params=neptun_params)
    neptune.log_artifact(io.StringIO(str(disc)), "disc.txt")
    neptune.log_artifact(io.StringIO(str(pure_gen)), "gen.txt")
    neptune.log_artifact(io.StringIO(str(list(init_gen_weights))), 'init_gen_weights.txt')
    neptune.log_artifact(io.StringIO(str(list(init_disc_weights))), 'init_disc_weights.txt')

In [ ]:
trainer = Trainer(g_values, 
                  data_bus_size, 
                  disc, 
                  gen, 
                  pure_gen,
                  real, 
                  pure_real,
                  out_qubit, 
                  ds, 
                  gs, 
                  real_symbols, 
                  ls,
                  real_values_provider = real_values_provider,
                  use_analytical_expectation=True,
                  use_neptune=use_neptune)

In [ ]:
snapshot_interval_epochs = 20
results, json_result = trainer.train(disc_weights,
      gen_weights, 
      opt, 
      epochs=epochs, 
      disc_iteration=disc_iteration, 
      gen_iteration=gen_iteration,
      snapshot_interval_epochs=snapshot_interval_epochs,
      early_stop_fidelity_threshold=None)
res_dict = trainer.get_params_and_results()

In [ ]:
trained_gen_pairs = res_dict['weights'][-1]['gen_pairs']
trained_disc_pairs = res_dict['weights'][-1]['disc_pairs']

In [ ]:
gen_evaluator = CircuitEvaluator(pure_gen, ls, trainer.label_value_provider, trained_gen_pairs, gs)
disc_evaluator = CircuitEvaluator(disc, ls, trainer.label_value_provider, trained_disc_pairs, ds)
real_evaluator = CircuitEvaluator(pure_real, real_symbols, trainer.real_values_provider)

In [ ]:
if use_neptune:
    neptune.log_artifact(io.StringIO(disc_evaluator.get_resolved_circuit().to_qasm()), 'desc_qasm.txt')
    neptune.log_artifact(io.StringIO(gen_evaluator.get_resolved_circuit().to_qasm()), 'gen_qasm.txt')
    neptune.log_artifact(io.StringIO(json_result), 'weights.json')

In [ ]:
def get_gen_for_g(g, gen_weights, gen_provider):
    rad = gen_provider(g)
    return np.append(gen_weights, rad)

def get_states_and_fidelty_for_real(gen_evaluator, 
                                    real_evaluator, 
                                    gen_pairs, 
                                    g, 
                                    sized):
    generated = gen_evaluator.get_state_from_params()
    real = real_evaluator.get_state_from_params(g)
    return generated, real, cirq.fidelity(generated, real)

def get_states_and_fidelty_for_ground(gen_evaluator, g, gen_weights, size):

    generated = gen_evaluator.get_state_from_params(trace_dims=list(range(size)))
    ground = get_ground_state_for_g(g, size)
    return generated, ground, cirq.fidelity(generated, ground)

def compare_generated_for_g(gen_evaluator, g1, g2, gen_weights, size):
    generated1 = gen_evaluator.get_state_from_params(trace_dims=list(range(size)))
    generated2 = gen_evaluator.get_state_from_params(trace_dims=list(range(size)))
    return generated1, generated2, cirq.fidelity(generated1, generated2)

In [ ]:
def angle_dist(g, phase_angles_provider, weights):
    angles = phase_angles_provider(g)
    z1_rad = 3 * math.pi / 2
    z2_rad = -3 * math.pi / 2
    z1_deg = math.degrees(z1_rad)
    z2_deg = math.degrees(z2_rad)
    
    xr_rad = angles[0]
    xv_rad = angles[1]
    xw_rad = angles[2]
    xr_deg = math.degrees(xr_rad)
    xv_deg = math.degrees(xv_rad)
    xw_deg = math.degrees(xw_rad)
    
    
    norm_w_deg = [math.degrees(w) for w in weights]
    deg_diff = [norm_w_deg[0], z1_deg, norm_w_deg[0] - z1_deg, weights[0], z1_rad]
    
    print(z1_deg,z2_deg,xr_deg,xv_deg,xw_deg)
    return deg_diff

In [ ]:
g = g_values[0]
generated_state, real_state, fidelity = \
    get_states_and_fidelty_for_real(gen_evaluator, real_evaluator, trained_gen_pairs, g, data_bus_size)

In [ ]:
generated_state, real_state, fidelity

In [ ]:
if use_neptune:
    neptune.log_artifact(io.StringIO(str({"generated_state": generated_state.tolist(), "real_state": real_state.tolist()})), f"states_g={g}.txt")

In [ ]:
if use_neptune:
    neptune.stop()

In [ ]:
get_g_parameters_provider()(g)

In [ ]:
real025 = real_evaluator.get_state_from_params(0.25)

In [ ]:
real04 = real_evaluator.get_state_from_params(0.4)
real01 = real_evaluator.get_state_from_params(0.1)

In [ ]:
cirq.fidelity(real025, -real025)

In [ ]:
compare_generated_for_g(gen_evaluator, -0.9, -0.8, trained_gen_weights, data_bus_size)

In [ ]:
g = 0
get_states_and_fidelty_for_ground(gen_evaluator, g, trained_gen_weights, data_bus_size)

In [243]:
q1, q2 = cirq.GridQubit.rect(1,2)

In [471]:
circuit = cirq.Circuit()
circuit.append([cirq.ZZPowGate(exponent=0.478, global_shift=-0.5).on(q1,q2)])
cirq.final_state_vector(circuit),circuit._unitary_()

(array([0.7311156-0.6822536j, 0.       +0.j       , 0.       +0.j       ,
        0.       +0.j       ], dtype=complex64),
 array([[0.73111559-0.68225361j, 0.        +0.j        ,
         0.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.73111559+0.68225361j,
         0.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.        +0.j        ,
         0.73111559+0.68225361j, 0.        +0.j        ],
        [0.        +0.j        , 0.        +0.j        ,
         0.        +0.j        , 0.73111559-0.68225361j]]))

In [464]:
circuit = cirq.Circuit()
circuit.append([cirq.ZZPowGate(exponent=2.25, global_shift=1).on(q1,q2)])
cirq.final_state_vector(circuit),circuit._unitary_()

(array([0.70710677+0.70710677j, 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ], dtype=complex64),
 array([[7.07106781e-01+0.70710678j, 0.00000000e+00+0.j        ,
         0.00000000e+00+0.j        , 0.00000000e+00+0.j        ],
        [0.00000000e+00+0.j        , 4.44089210e-16+1.j        ,
         0.00000000e+00+0.j        , 0.00000000e+00+0.j        ],
        [0.00000000e+00+0.j        , 0.00000000e+00+0.j        ,
         4.44089210e-16+1.j        , 0.00000000e+00+0.j        ],
        [0.00000000e+00+0.j        , 0.00000000e+00+0.j        ,
         0.00000000e+00+0.j        , 7.07106781e-01+0.70710678j]]))